In [158]:
import csv

In [159]:
class Cell:
    def __init__(self, params_file):
        self.params = {}
        self.load_file(params_file)
    
    def load_file(self, params_file):
        reader = csv.reader(open(params_file))        
        for row in reader:
            param = row[0]
            value = float(row[1])
            self.params[param] = value
        self.calc_base_params()
        self.calc_x_thermal_resistance()
        self.calc_y_thermal_resistance()
        self.calc_z1_thermal_resistance()
        self.calc_z2_thermal_resistance()
        self.calc_heat_flow()
    
    def set_params(self, param, value):
        self.params[param] = value
    
    def calc_base_params(self):
        self.params["jellyroll_x_dimension"] = self.params["cell_x_dimension"] - 2*self.params["cell_can_thickness"]
        self.params["jellyroll_y_dimension"] = self.params["cell_y_dimension"] - (2*self.params["cell_can_thickness"] + 2*self.params["cell_sidespace"])
        self.params["jellyroll_z_dimension"] = self.params["cell_z_dimension"] - (2*self.params["cell_can_thickness"] + self.params["cell_tailspace"] + self.params["cell_headspace"])
        self.params["pos_neck_y"] = 0.2*self.params["jellyroll_y_dimension"]
        self.params["neg_neck_y"] = 0.2*self.params["jellyroll_y_dimension"]
    
    def calc_x_thermal_resistance(self):        
        #jellyroll thermal resistance
        Rkx1 = self.params["jellyroll_x_dimension"]/2/(self.params["jellyroll_y_dimension"]*self.params["jellyroll_z_dimension"]*self.params["jellyroll_x_k"])
        #cellcan thermal resistance
        Rkx2 = self.params["cell_can_thickness"]/(self.params["cell_y_dimension"]*self.params["cell_z_dimension"]*self.params["cell_can_k"])
        self.Rkx = [Rkx1, Rkx2]
    
    def calc_y_thermal_resistance(self):        
        #jellyroll thermal resistance
        Rky1 = self.params["jellyroll_y_dimension"]/2/(self.params["jellyroll_x_dimension"]*self.params["jellyroll_z_dimension"]*self.params["jellyroll_y_k"])
        #airgap thermal resistance
        Rky2 = self.params["cell_sidespace"]/(self.params["cell_x_dimension"]*self.params["cell_z_dimension"]*self.params["air_k"])
        #cellcan thermal resistance
        Rky3 = self.params["cell_can_thickness"]/(self.params["cell_x_dimension"]*self.params["cell_z_dimension"]*self.params["cell_can_k"])
        self.Rky = [Rky1, Rky2, Rky3]
    
    def calc_z1_thermal_resistance(self):        
        #jellyroll thermal resistance
        Rkz11 = self.params["jellyroll_z_dimension"]/2/(self.params["jellyroll_x_dimension"]*self.params["jellyroll_y_dimension"]*self.params["jellyroll_z_k"])                
        #neck thermal resistance
        Rkz12_pos = self.params["pos_neck_z"]/(self.params["pos_neck_x"]*self.params["pos_neck_y"]*self.params["pos_neck_k"])
        Rkz12_neg = self.params["neg_neck_z"]/(self.params["neg_neck_x"]*self.params["neg_neck_y"]*self.params["neg_neck_k"])
        #celltab thermal resistance
        Rkz13_pos = self.params["pos_tab_z"]/(self.params["pos_tab_x"]*self.params["pos_tab_y"]*self.params["pos_tab_k"])
        Rkz13_neg = self.params["neg_tab_z"]/(self.params["neg_tab_x"]*self.params["neg_tab_y"]*self.params["neg_tab_k"])
        self.Rkz1 = [Rkz11, Rkz12_pos, Rkz12_neg, Rkz13_pos, Rkz13_neg]
    
    def calc_z2_thermal_resistance(self):
        #jellyroll thermal resistance
        Rkz21 = self.params["jellyroll_z_dimension"]/2/(self.params["jellyroll_x_dimension"]*self.params["jellyroll_y_dimension"]*self.params["jellyroll_z_k"])
        #airgap thermal resistance
        Rkz22 = self.params["cell_tailspace"]/(self.params["cell_x_dimension"]*self.params["cell_y_dimension"]*self.params["air_k"])
        #cellcan thermal resistance
        Rkz23 = self.params["cell_can_thickness"]/(self.params["cell_x_dimension"]*self.params["cell_y_dimension"]*self.params["cell_can_k"])
        self.Rkz2 = [Rkz21, Rkz22, Rkz23]
    
    def calc_heat_flow(self):
        self.Rx1 = sum(self.Rkx)
        self.Rx2 = sum(self.Rkx)
        self.Ry1 = sum(self.Rky)
        self.Ry2 = sum(self.Rky)
        self.Rz1_pos = self.Rkz1[0] + self.Rkz1[1] + self.Rkz1[3]
        self.Rz1_neg = self.Rkz1[0] + self.Rkz1[2] + self.Rkz1[4]
        self.Rz2 = sum(self.Rkz2)
        self.R = [self.Rx1, self.Rx2, self.Ry1, self.Ry2, self.Rz1_pos, self.Rz1_neg, self.Rz2]

In [160]:
class Cooling_X:
    def __init__(self, cell):
        self.cell = cell

    def define_h(self,h, T_inf):
        self.h = h
        self.T_inf = T_inf
        self.calc_resistance()

    def calc_resistance(self):
        self.Rx1 = 1/(self.h*self.cell.params["cell_y_dimension"]*self.cell.params["cell_z_dimension"])
        self.Rx2 = 1/(self.h*self.cell.params["cell_y_dimension"]*self.cell.params["cell_z_dimension"])
        self.Rx = [self.Rx1, self.Rx2]

In [161]:
class Cooling_Y:
    def __init__(self, cell):
        self.cell = cell

    def define_h(self,h, T_inf):
        self.h = h
        self.T_inf = T_inf
        self.calc_resistance()

    def calc_resistance(self):
        self.Ry1 = 1/(self.h*self.cell.params["cell_x_dimension"]*self.cell.params["cell_z_dimension"])
        self.Ry2 = 1/(self.h*self.cell.params["cell_x_dimension"]*self.cell.params["cell_z_dimension"])
        self.Ry = [self.Ry1, self.Ry2]

In [162]:
class Cooling_Z1:
    def __init__(self, cell):
        self.cell = cell

    def define_h(self,h, T_inf):
        self.h = h        
        self.T_inf = T_inf
        self.calc_resistance()

    def calc_resistance(self):
        self.Rz1_pos = 1/(self.h*self.cell.params["pos_tab_x"]*self.cell.params["pos_tab_y"])        
        self.Rz1_neg = 1/(self.h*self.cell.params["neg_tab_x"]*self.cell.params["neg_tab_y"])        
        self.Rz1 = [self.Rz1_pos, self.Rz1_neg]

In [163]:
class Cooling_Z2:
    def __init__(self, cell):
        self.cell = cell

    def define_h(self,h, T_inf):
        self.h = h        
        self.T_inf = T_inf
        self.calc_resistance()

    def calc_resistance(self):
        self.Rz2 = 1/(self.h*self.cell.params["cell_x_dimension"]*self.cell.params["cell_y_dimension"])                
        self.Rz = [self.Rz2]

In [164]:
class System:
    def __init__(self, cell, cooling_x, cooling_y, cooling_z1, cooling_z2): 
        self.cell = cell
        self.cooling_x = cooling_x
        self.cooling_y = cooling_y
        self.cooling_z1 = cooling_z1
        self.cooling_z2 = cooling_z2       
        self.R_system = [self.cell.Rx1+self.cooling_x.Rx1, self.cell.Rx2+self.cooling_x.Rx2,
                         self.cell.Ry1+self.cooling_y.Ry1, self.cell.Ry2+self.cooling_y.Ry2,
                         self.cell.Rz1_pos+self.cooling_z1.Rz1_pos, self.cell.Rz1_neg+self.cooling_z1.Rz1_neg,
                         self.cell.Rz2+self.cooling_z2.Rz2]
        self.calculate_heat_dist()
        
    def calculate_heat_dist(self):
        temp = []
        for i in self.R_system:
            temp.append(1/i)
        q_sum = sum(temp)
        self.q_dist = []
        for i in temp:
            self.q_dist.append(i/q_sum)
            
    def define_initial_temperature(self, initial_temperature):
        self.T_init = initial_temperature        

    def define_heat_generation(self,heat_generated):
        self.q = heat_generated
    
    def run_simulation(self,nt):
        T_core, Tx1, Tx2, Ty1, Ty2, Tz1_pos, Tz1_neg, Tz2 = [self.T_init],[self.T_init],[self.T_init],[self.T_init],[self.T_init],[self.T_init],[self.T_init],[self.T_init]
        for step in range(nt):            
            T_core.append(T_core[-1] + (self.q/(self.cell.params["cell_mass"]*self.cell.params["cell_cp"])))
            Tx1.append(T_core[-1] - self.R_system[0]*self.q_dist[0]*self.q)
            Tx2.append(T_core[-1] - self.R_system[1]*self.q_dist[1]*self.q)
            Ty1.append(T_core[-1] - self.R_system[2]*self.q_dist[2]*self.q)
            Ty2.append(T_core[-1] - self.R_system[3]*self.q_dist[3]*self.q)
            Tz1_pos.append(T_core[-1] - self.R_system[4]*self.q_dist[4]*self.q)
            Tz1_neg.append(T_core[-1] - self.R_system[5]*self.q_dist[5]*self.q)
            Tz2.append(T_core[-1] - self.R_system[6]*self.q_dist[6]*self.q)
        self.T_core = T_core
        self.Tx1, self.Tx2, self.Ty1, self.Ty2, self.Tz1_pos, self.Tz1_neg, self.Tz2 = Tx1, Tx2, Ty1, Ty2, Tz1_pos, Tz1_neg, Tz2

In [165]:
params_file = r"/home/kage-main/Documents/01_Codes/02_projects/Cell_SOC_Model/helper_files/cell_thermal_model.csv"
cell = Cell(params_file)
cooling_x = Cooling_X(cell)
cooling_x.define_h(5,298)
cooling_y = Cooling_Y(cell)
cooling_y.define_h(5,298)
cooling_z1 = Cooling_Z1(cell)
cooling_z1.define_h(200,298)
cooling_z2 = Cooling_Z2(cell)
cooling_z2.define_h(200,298)
system = System(cell, cooling_x, cooling_y, cooling_z1, cooling_z2)
system.define_initial_temperature(300)
system.define_heat_generation(0.65)
system.run_simulation(3600)

In [168]:
system.Tx1

[300,
 297.6045970823948,
 297.6056171702793,
 297.6066372581638,
 297.6076573460483,
 297.6086774339328,
 297.60969752181734,
 297.61071760970185,
 297.61173769758636,
 297.6127577854709,
 297.6137778733554,
 297.6147979612399,
 297.6158180491244,
 297.6168381370089,
 297.61785822489344,
 297.61887831277795,
 297.61989840066246,
 297.620918488547,
 297.6219385764315,
 297.622958664316,
 297.6239787522005,
 297.624998840085,
 297.62601892796954,
 297.62703901585405,
 297.62805910373856,
 297.6290791916231,
 297.6300992795076,
 297.6311193673921,
 297.6321394552766,
 297.6331595431611,
 297.63417963104564,
 297.63519971893015,
 297.63621980681467,
 297.6372398946992,
 297.6382599825837,
 297.6392800704682,
 297.6403001583527,
 297.6413202462372,
 297.64234033412174,
 297.64336042200625,
 297.64438050989077,
 297.6454005977753,
 297.6464206856598,
 297.6474407735443,
 297.6484608614288,
 297.64948094931333,
 297.65050103719784,
 297.65152112508235,
 297.65254121296687,
 297.6535613008514